In [1]:
%matplotlib inline

This is a short example of how the DNA agarose gel electrophoresis in pydna can be used.
The %matplotlib inline in the line above is there to facilitate viewing matplotlib plots in    

## Pydna Gel Electrophoresis Simulation

In [2]:
from pydna.gel import weight_standard_sample, Gel

In [3]:
st = weight_standard_sample('1kb+_GeneRuler')

In [4]:
from pydna.genbank import Genbank

In [5]:
gb = Genbank("bjornjobb@gmail.com")

In [6]:
gene = gb.nucleotide("X06997")

In [7]:
from pydna.parsers import parse_primers

In [8]:
primer_f, primer_r =      parse_primers(''' >760_KlLAC12_rv (20-mer)
                                             ttaaacagattctgcctctg

                                             >759_KlLAC12_fw (19-mer)
                                             aaatggcagatcattcgag
                                         ''')

In [9]:
from pydna.amplify import pcr

In [10]:
# PCR
pcr_prod = pcr(primer_f, primer_r, gene)

In [11]:
pcr_prod

Amplicon(1766)

In [12]:
Gel([ st,[pcr_prod]] , gel_len=16).run()

TypeError: no implementation found for 'numpy.broadcast_arrays' on types that implement __array_function__: [<class 'pint.quantity.build_quantity_class.<locals>.Quantity'>]